In [1]:
# Recommended Kernel to use: Python 3.11.5
%pip install pandas     
%pip install numpy
%pip install opencv-python-headless
%pip install protobuf
%pip install tensorflow
%pip install keras
%pip install scikit-learn
%pip install labelImg
%pip install tensorflow-object-detection-api

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)
Note: you may need to restart the kernel to use updated packages.

  Using cached protobuf-4.25.3-cp38-cp38-win_amd64.whl (413 kB)
  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl (276.5 MB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached tensorflow_io_g

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
spyder 4.2.5 requires jedi==0.17.2, but you have jedi 0.19.1 which is incompatible.
spyder 4.2.5 requires parso==0.7.0, but you have parso 0.8.3 which is incompatible.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.3 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
Note: you may need to restart the kernel to use updated packages.
  Using cached labelImg-1.8.6.tar.gz (247 kB)
  Using cached PyQt5-5.15.10-cp37-abi3-win_amd64.whl (6.8 MB)
  Using cached PyQt5_sip-12.13.0-cp38-cp38-win_amd64.whl (78 kB)
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
  Created wheel for labelImg: filename=labelImg-1.8.6-py2.py3-none-any.whl size=261529 sha256=bc4a6d257fdd33f225245922470afd13f03350e3e4e32ce47e0d8bf8be818dc6
  Stored in directory: c:\users\dariu\appdata\local\pip\cache\wheels\c3\9e\49\8368f5bc5347b5e54aef95b7b03ec56af7e23ea4c16c82109c
Successfully built labelImg
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
spyder 4.2.5 requires jedi==0.17.2, but you have jedi 0.19.1 which is incompatible.
spyder 4.2.5 requires parso==0.7.0, but you have parso 0.8.3 which is incompatible.
spyder 4.2.5 requires pyqt5<5.13, but you have pyqt5 5.15.10 which is incompatible.


  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844513 sha256=0414d0352a0ea9a6cc907c99b5d9b4d51ade005c27abca07c9d68b3d0c66fbca
  Stored in directory: c:\users\dariu\appdata\local\pip\cache\wheels\93\4e\cd\e2d0aecb1680cbc6730933bc53a776f7213acb08ecbb91c301
Successfully built tensorflow-object-detection-api
  Attempting uninstall: pkginfo
    Found existing installation: pkginfo 1.7.0
    Uninstalling pkginfo-1.7.0:
      Successfully uninstalled pkginfo-1.7.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
    Standard Python Machine Learning Libraries, for array structure and dataset loading
"""
import pandas as pd
import numpy as np


"""
    MediaPipe - used for handlandmark detection
    OpenCv - Video Image Data Extraction
    JSON | Protobuf - Data Formating
"""
import cv2
import json
from google.protobuf.json_format import MessageToJson


"""
    Libraries for file name processing and preparation
"""
import random
import string
import shutil
import os


"""
    Libraries to keep sequence consistency
"""
from keras.preprocessing.sequence import pad_sequences
import math

"""
    Libraries for splitting training and testing data and also transform our label classes
"""

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


"""
    Neural Network Libraries
"""

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, BatchNormalization, Activation
from keras.callbacks import LearningRateScheduler

### Only Run the Code Below if frames are not yet generated from the given clips.

In [3]:
"""
    Utility Function for UID generation and renaming of files
"""

def generate_random_uid():
    """Generate a random UID."""
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))



"""
    Remake files for class labeling and converting videos to frames
"""
def rename_videos(source_directory, destination_directory, class_names, idx):
    """Rename videos in the specified directory."""
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            if file.endswith(".MOV"):
                old_path = os.path.join(root, file)

                # Convert .MOV to frames
                vidcap = cv2.VideoCapture(old_path)

                # Get the frame rate and calculate the number of frames to skip
                fps = vidcap.get(cv2.CAP_PROP_FPS)
                skip_frames_start = int(2 * fps)

                # Get the total number of frames and calculate the duration of the video in seconds
                total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
                video_duration = total_frames / fps

                # Calculate the timestamp to stop (2 seconds before the end)
                stop_timestamp = video_duration - 1.95

                # Convert the timestamp to frame index
                stop_frame_index = int(stop_timestamp * fps)

                success, image = vidcap.read()
                count = 0
                saved_frames_count = 0
                while success and count < stop_frame_index:
                    if count >= skip_frames_start:
                        # f"{class_names[idx]}"
                        frame_name = os.path.join(destination_directory, f"{class_names[idx]}_{generate_random_uid()}.jpg")
                        os.makedirs(os.path.dirname(frame_name), exist_ok=True)
                        cv2.imwrite(frame_name, image)     # save frame as JPEG file
                        saved_frames_count += 1
                    success, image = vidcap.read()
                    count += 1

                print(f"Converted {file} to {saved_frames_count} frames")
                
""" 
    Update Target Classes Filenames and move them to a single directory for dataset processing
"""
# Specify the source directory containing the videos
source_directory = './dataset/clips/'

# Specify the destination directory for the renamed videos
destination_directory = './objectdetection/workspace/images/od_data/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Call the function to rename and copy the videos
targetSets = [85, 86, 89, 94]

# Specify the class names for each index
class_names = ['bread', 'egg', 'chicken', 'crab']

for idx, target in enumerate(targetSets):
    print("Renaming Target Set: ", target)
    rename_videos(source_directory + str(target), destination_directory, class_names, idx)
    print("\n\n")


Renaming Target Set:  85
Converted 11taoceh_0.MOV to 6 frames
Converted 4asdlwz1_0.MOV to 6 frames
Converted 4wjsa5wq_0.MOV to 7 frames
Converted 940bqr4z_0.MOV to 6 frames
Converted 9dppbec3_0.MOV to 7 frames
Converted f9r9bsva_0.MOV to 6 frames
Converted fdpnld0t_0.MOV to 6 frames
Converted fx5jgfby_0.MOV to 7 frames
Converted gwa534to_0.MOV to 6 frames
Converted i6641z1q_0.MOV to 7 frames
Converted inl5n07c_0.MOV to 7 frames
Converted mato6gvs_0.MOV to 6 frames
Converted nz7q913t_0.MOV to 6 frames
Converted opunh4zs_0.MOV to 6 frames
Converted p5ga19wa_0.MOV to 6 frames
Converted qygo7g5s_0.MOV to 6 frames
Converted r4znhltx_0.MOV to 7 frames
Converted titl2dvs_0.MOV to 6 frames
Converted tzd3o8eu_0.MOV to 6 frames
Converted vvzajti5_0.MOV to 6 frames



Renaming Target Set:  86
Converted 0.MOV to 6 frames
Converted 1.MOV to 6 frames
Converted 10.MOV to 6 frames
Converted 11.MOV to 6 frames
Converted 12.MOV to 6 frames
Converted 13.MOV to 6 frames
Converted 14.MOV to 6 frames
Conver

### After generating the frames from the clips, label each frame using labelImg manually

### Run the code below to split the dataset into train and test data with an 80:20 ratio

In [5]:
# Specify the directory containing the image-xml pairs
data_directory = './objectdetection/workspace/images/od_data/'

# Specify the directories for the train and test sets
train_directory = './objectdetection/workspace/images/train/'
test_directory = './objectdetection/workspace/images/test/'

# Create the train and test directories if they don't exist
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# Get the list of image files
image_files = [file for file in os.listdir(data_directory) if file.endswith('.jpg')]

# Shuffle the list of files randomly
random.shuffle(image_files)

# Calculate the number of files for the train and test sets
total_files = len(image_files)
train_split = int(0.8 * total_files)
test_split = total_files - train_split

# Split the files into train and test sets
train_files = image_files[:train_split]
test_files = image_files[train_split:]

# Counter for the number of files
train_count = 0
test_count = 0

# Copy the train files to the train directory
for file in train_files:
    xml_file = os.path.splitext(file)[0] + '.xml'
    shutil.copy(os.path.join(data_directory, file), os.path.join(train_directory, file))
    shutil.copy(os.path.join(data_directory, xml_file), os.path.join(train_directory, xml_file))
    train_count+= 1

# Copy the test files to the test directory
for file in test_files:
    xml_file = os.path.splitext(file)[0] + '.xml'
    shutil.copy(os.path.join(data_directory, file), os.path.join(test_directory, file))
    shutil.copy(os.path.join(data_directory, xml_file), os.path.join(test_directory, xml_file))
    test_count += 1

print(f"Train directory created with {train_count} images, with a total of {train_count * 2} files.")
print(f"Test directory created with {test_count} images, with a total of {test_count * 2} files.")
print(f"Train/Test Ratio: {round(train_count / (train_count + test_count) * 100, 2)}:{round(test_count / (train_count + test_count) * 100, 2)}")

Train directory created with 330 images, with a total of 660 files.
Test directory created with 83 images, with a total of 166 files.
Train/Test Ratio: 79.9:20.1
